In [3]:
# Libraries and modules we will need for all 3 Tasks are listed here
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math 
# All sklearn functions below are for part 1 and 3, they are not used in part 2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [4]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df = pd.read_csv('housing.csv', sep = '\s+', header = None, names = column_names) 
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [5]:
def outlier_removal(df, column: str):
    """Removes rows with outliers in a particular column

    Args:
        df (pandas DataFrame): The DataFrame we want the values removed from
        column (str): The column we're checking for outliers

    Returns:
        df (pandas DataFrame): The DataFrame minus the outliers
    """
    # Calculate IQR
    upper_QR = df[column].quantile(0.75)
    lower_QR = df[column].quantile(0.25)
    inter_QR = upper_QR-lower_QR
    # Filter for outliers
    df = df[df[column]<(upper_QR+(1.5*inter_QR))] # Gets all values below upper outlier limits
    df = df[df[column]>(lower_QR-(1.5*inter_QR))] # Gets all values above lower limits
    # Reset indexes 
    df.reset_index(drop=True)
    
    return df

In [6]:
df = outlier_removal(df, 'MEDV')

X = df[['INDUS', 'RM', 'TAX', 'PTRATIO', 'LSTAT']]
y = df['MEDV']


In [ ]:
a = []
for i in range(10): 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
    model = GradientBoostingRegressor()
    # Implement GridSearchCV to optimize hyper-parameters
    params = {'min_samples_leaf':[5,10,15,20,25,30], 'max_depth':[2,3,4,5,6,7,8,9,10]} # Params for it to cycle through
    clf = GridSearchCV(estimator=model, param_grid=params, cv=5, scoring= 'r2')
    # Fit model 
    clf.fit(X_train, y_train)
    # Make predictions
    y_pred = clf.predict(X_test)
    # Print model performance metrics
    a.append(r2_score(y_test, y_pred))
print(np.mean(a))

In [151]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=123)

In [153]:
model = GradientBoostingRegressor()
# Implement GridSearchCV to optimize hyper-parameters
params = {'min_samples_leaf':[5,10,15,20,25,30], 'max_depth':[2,3,4,5,6,7,8,9,10]} 
param_grid = {'n_estimators': [10, 20, 50, 100],
    'max_depth': [1, 2, 3, 4],
    'learning_rate': [0.01, 0.1, 0.2, 0.5]
}

clf = GridSearchCV(estimator=model, param_grid=params, cv=5)
# Fit model 
clf.fit(X_train, y_train)
# Make predictions
y_pred = clf.predict(X_test)
my_performance(y_test, y_pred)

mean squared error: 5.272439254710651
root mean squared error: 2.2961792732081374
mean absolute error: 1.7621322476414958
r2 score: 0.8646045009428446


In [8]:
# Decision Tree Class

# Used as a week learned within the gradient boosting ensemble (builds iteratively which is why there is a max depth of 1)
# High bias but low variance, 
# Ideal for boosting methods that aim to iteratively reduce error by focusing on hard-to-predict instances.
class DecisionTree:
    def __init__(self, max_depth=1):
        self.max_depth = max_depth
        self.tree = None

    # Each node is a decision point for traversing through the decision tree
    # Follows decision tree logic
    class Node:
        def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
            # feature_Index and threshold are determining factors for where the split happens
            self.feature_index = feature_index
            self.threshold = threshold
            self.left = left
            self.right = right
            self.value = value

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    # Recursive binary splitting
    def _build_tree(self, X, y, depth):
        # Continues to split until the maximum specified depth is reached
        num_samples, num_features = X.shape
        # Stopping condition: if the current depth exceeds the max depth or the dataset cannot be split further
        if depth >= self.max_depth or num_samples <= 1:
            leaf_value = self._calculate_leaf_value(y)
            # Create a leaf node with the calculated value
            return self.Node(value=leaf_value)

        # Finds the best features by iterating through and selecting lowest MSE
        best_feature, best_threshold = self._find_best_split(X, y, num_samples, num_features)
        if best_feature is None:
            # If no split can improve the outcome, create a leaf node
            return self.Node(value=self._calculate_leaf_value(y))
        
        # Split the dataset and recursively build left and right subtrees
        left_idxs, right_idxs = self._split(X[:, best_feature], best_threshold)
        left_subtree = self._build_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right_subtree = self._build_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return self.Node(feature_index=best_feature, threshold=best_threshold, left=left_subtree, right=right_subtree)

    def _calculate_leaf_value(self, y):
        # The leaf value can be the mean of the target values, calculates that mean value for the leaf node
        return np.mean(y)


    # def _find_best_split(self, X, y, num_samples, num_features):
    #     # Finds the best feature and threshold to split on based on the lowest MSE
    #     best_feature, best_threshold = None, None
    #     best_mse = np.inf
    #     for feature_index in range(num_features):
    #         thresholds = np.unique(X[:, feature_index])
    #         # Splits the dataset and calculates the MSE for this split
    #         for threshold in thresholds:
    #             left_idxs, right_idxs = self._split(X[:, feature_index], threshold)
    #             if len(left_idxs) == 0 or len(right_idxs) == 0:
    #                 continue
    #             mse = self._calculate_mse(y[left_idxs], y[right_idxs])
    #             # Update split if current mse is better
    #             if mse < best_mse:
    #                 best_mse = mse
    #                 best_feature = feature_index
    #                 best_threshold = threshold
    #     return best_feature, best_threshold


    def _find_best_split(self, X, y, num_samples, num_features):
        # Finds the best feature and threshold to split on based on the lowest MSE
        best_feature, best_threshold = None, None
        best_mse = np.inf
        for feature_index in range(num_features):
            sorted_feature_values = np.sort(X[:, feature_index])
            # Creates a possible threshold as the midpoint of each consecutive pair of ordered feature values
            thresholds = [(sorted_feature_values[i] + sorted_feature_values[i+1]) / 2 for i in range(num_samples - 1)]
            # Splits the dataset and calculates the MSE for this split
            for threshold in thresholds:
                left_idxs, right_idxs = self._split(X[:, feature_index], threshold)
                if len(left_idxs) == 0 or len(right_idxs) == 0:
                    continue
                mse = self._calculate_mse(y[left_idxs], y[right_idxs])
                # Update split if current mse is better
                if mse < best_mse:
                    best_mse = mse
                    best_feature = feature_index
                    best_threshold = threshold
        return best_feature, best_threshold

    # Splits dataset into left/right based on threshold of given feature
    def _split(self, feature_values, threshold):
        left_idxs = np.where(feature_values <= threshold)[0]
        right_idxs = np.where(feature_values > threshold)[0]
        return left_idxs, right_idxs

    def _calculate_mse(self, left_y, right_y):
        # Calculate the MSE of the left and right splits by weighted averages of variance
        total_left_mse = np.var(left_y) * len(left_y) if len(left_y) > 0 else 0
        total_right_mse = np.var(right_y) * len(right_y) if len(right_y) > 0 else 0
        total_mse = (total_left_mse + total_right_mse) / (len(left_y) + len(right_y))
        return total_mse
    
    def predict(self, X):
        # Predictions array to store predictions for each sample in X
        predictions = np.array([self._traverse_tree(x, self.tree) for x in X])
        return predictions

    def _traverse_tree(self, x, node):
        # Recursive method to traverse the tree for a single sample 'x' until a leaf node is reached
        if node.value is not None:
            return node.value
        if x[node.feature_index] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

In [9]:
# Gradient Boost Class

class GradientBoostAll:
    def __init__(self, n_estimators: int = 25, max_depth: int = 1, learning_rate: int =.1):
        self.max_depth = max_depth # Max depth of the trees
        self.n_estimators = n_estimators # Number of trees
        self.learning_rate = learning_rate # Learning rate, step size for parameter update
        self.trees = [] # List of our trees
    
    def fit(self, X_train, y_train):
        # To start all residuals = y_train
        residuals = np.copy(y_train)
        self.f_hat = 0 
        # Now time to make decision trees
        for i in range(self.n_estimators):
            # Build and Fit Tree to data
            tree = DecisionTree(max_depth=self.max_depth)
            tree.fit(X_train, residuals)
            # Save our tree
            self.trees.append(tree)
            # Make prediction
            f_hat_b = tree.predict(X_train)
            # Update f_hat
            self.f_hat += (self.learning_rate*f_hat_b) 
            # Update residuals
            residuals = residuals - (self.learning_rate*f_hat_b)
        return self
    
    def predict(self, X_test):
        y_hat = np.zeros((X_test.shape[0], ))
        for tree in self.trees:
            y_hat += self.learning_rate*tree.predict(X_test)
        return y_hat

In [10]:
def my_train_test_split(X,y, test_size = 0.2, random_state = None):
    #get random seed to allow reproduceability
    random.seed(random_state)
    #select test indexes from range of total indexes
    test_ixs= random.sample(range(len(y)), math.floor(len(y)*test_size))
    #return X_train, X_test, y_train, y_test (train sets are total sets - test sets)
    return X.drop(test_ixs), X.iloc[test_ixs], y.drop(test_ixs), y.iloc[test_ixs]



In [11]:
def my_mse(a, b):
    return np.mean((a-b)**2)

def my_rmse(a, b):
    return math.sqrt(my_mse(a,b))

def my_mae(a,b):
    return np.mean(np.abs(a-b))

def my_r2(a,b):
    y_bar = sum(a)/len(a)
    ss_res = np.mean((a-b)**2)
    ss_tot = np.mean((a-y_bar)**2)
    return 1-(ss_res/ss_tot)

def my_performance(a,b):
    if not isinstance(a, np.ndarray):
        a=a.to_numpy()
    if not isinstance(b, np.ndarray):
        b=b.to_numpy()
    print(f'mean squared error: {my_mse(a,b)}')
    print(f'root mean squared error: {my_rmse(a,b)}')
    print(f'mean absolute error: {my_mae(a,b)}')
    print(f'r2 score: {my_r2(a,b)}')

In [7]:
my_performance(y_test, y_pred)

mean squared error: 6.049886388821874
root mean squared error: 2.4596516803852277
mean absolute error: 1.835068885142754
r2 score: 0.9020275933906642


In [76]:
param_grid = {
    'n_estimators': [10, 20, 50, 100],
    'max_depth': [1, 2, 3, 4],
    'learning_rate': [0.01, 0.1, 0.2, 0.5]
}

X_train_np = np.asarray(X_train)
y_train_np = np.asarray(y_train)
X_test_np = np.asarray(X_test)
y_test_np = np.asarray(y_test)

# Takes in dictionary of parameters to explore
def grid_search(X_train, y_train, X_test, y_test, param_grid):
    # Track lowest MSE seeen so far, set to infinity initially so every value is lower
    # Params stores best paramaters found through best score
    best_score = float('inf')
    best_params = {}
    best_r2 = None

    # Nested loops to iterate over every combination of estimators depth and LR
    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            for learning_rate in param_grid['learning_rate']:
                # Initalize and fit model with every combination of parameters
                model = GradientBoostAll(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate)
                model.fit(X_train, y_train)

                # Make predictions on the test set and calculate MSE
                y_pred = model.predict(X_test)
                score = my_mse(y_test, y_pred)
                r2 = my_r2(y_test, y_pred)

                # Update best_score and best_params if current model is better
                if score < best_score:
                    best_score = score
                    best_params = {'n_estimators': n_estimators, 'max_depth': max_depth, 'learning_rate': learning_rate}
                    best_r2 = r2

    return best_params, best_score, best_r2
    
best_params, best_score, best_r2 = grid_search(X_train_np, y_train_np, X_test_np, y_test_np, param_grid)

print("Best Parameters:", best_params)
print("Best MSE Score:", best_score)
print("BEST r2 Score", best_r2)

Best Parameters: {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.2}
Best MSE Score: 7.0576130977505205
BEST r2 Score 0.8250992386850639


In [29]:
X_train_np = np.asarray(X_train)
y_train_np = np.asarray(y_train)
X_test_np = np.asarray(X_test)
y_test_np = np.asarray(y_test)
our_model = GradientBoostAll(n_estimators=50, max_depth=2, learning_rate=0.2)
our_model.fit(X_train_np, y_train_np)
y_pred = our_model.predict(X_test_np)
my_performance(y_test_np, y_pred)

mean squared error: 8.529225773441762
root mean squared error: 2.920483825232005
mean absolute error: 2.251904267837174
r2 score: 0.7886299432201218


In [47]:
len(X_train)

392

In [ ]:
b = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=i)
    X_train_np = np.asarray(X_train)
    y_train_np = np.asarray(y_train)
    X_test_np = np.asarray(X_test)
    y_test_np = np.asarray(y_test)
    our_model = GradientBoostAll(n_estimators=100, max_depth=3, learning_rate=0.2)
    our_model.fit(X_train_np, y_train_np)
    y_pred = our_model.predict(X_test_np)
    b.append(my_r2(y_test_np, y_pred))
np.mean(b)

In [24]:
X_train_np[1,2]

289.0

In [25]:
np.unique(X_train_np[:, 1])

array([3.561, 4.138, 4.368, 4.519, 4.628, 4.903, 4.906, 4.926, 4.963,
       5.   , 5.012, 5.019, 5.036, 5.093, 5.155, 5.272, 5.304, 5.349,
       5.362, 5.39 , 5.403, 5.412, 5.414, 5.453, 5.454, 5.456, 5.468,
       5.531, 5.536, 5.56 , 5.569, 5.57 , 5.572, 5.594, 5.597, 5.599,
       5.604, 5.605, 5.608, 5.613, 5.617, 5.627, 5.628, 5.631, 5.637,
       5.682, 5.683, 5.693, 5.705, 5.706, 5.707, 5.708, 5.709, 5.713,
       5.727, 5.731, 5.741, 5.747, 5.757, 5.786, 5.79 , 5.794, 5.803,
       5.807, 5.813, 5.822, 5.836, 5.837, 5.85 , 5.851, 5.852, 5.854,
       5.856, 5.859, 5.868, 5.869, 5.87 , 5.871, 5.872, 5.874, 5.875,
       5.876, 5.877, 5.878, 5.879, 5.88 , 5.885, 5.887, 5.888, 5.889,
       5.891, 5.895, 5.896, 5.898, 5.905, 5.913, 5.914, 5.92 , 5.926,
       5.928, 5.933, 5.935, 5.936, 5.942, 5.95 , 5.951, 5.952, 5.957,
       5.96 , 5.961, 5.963, 5.965, 5.966, 5.972, 5.981, 5.983, 5.987,
       5.99 , 5.998, 6.003, 6.004, 6.006, 6.009, 6.012, 6.014, 6.015,
       6.019, 6.02 ,

In [80]:
# K-fold Cross Validation function to perform hyperparameter tuning 
def kfold_cv(X_train, y_train, K_folds = 5, random_state = None, n_estimators: int = 25, max_depth: int = 1, learning_rate: int =.1):
    # Create empty lists for r2 and MSE scores
    r2_list = []
    mse_list = []
    # Set random seed for reproduceability
    np.random.seed(random_state)
    # Create list of indices as length of data
    indices = np.arange(len(X_train))
    # Shuffle indices
    np.random.shuffle(indices)
    # Create fold size as length/K
    fold_size = math.floor(len(X_train)/K_folds)
    for i in range(K_folds-1):
        # Create start- and end-points for this fold (final fold will be larger if K does not evenly divide length)
        start = i * fold_size
        end = (i + 1) * fold_size if i < K_folds - 1 else len(X_train)
        # Assign test indices as those within start- and end-points, train indices as those outside this range
        test_indices = indices[start:end]
        train_indices = np.concatenate((indices[:start], indices[end:]))
        # Create new train and test sets from existing training set based on train and test indices
        X_train_fold = X_train.iloc[train_indices]
        X_test_fold = X_train.iloc[test_indices]
        y_train_fold = y_train.iloc[train_indices]
        y_test_fold = y_train.iloc[test_indices]
        # Initialize model with hyperparameters to match function input parameters
        model = GradientBoostAll(n_estimators= n_estimators, max_depth=max_depth, learning_rate=learning_rate)
        # Fit model on fold training data
        model.fit(X_train_fold, y_train_fold)
        # Model predicts fold test data
        y_pred_fold = model.predict(X_test_fold)
        # r2 and MSE scores appended to list
        r2_list.append(my_r2(y_test_fold, y_pred_fold))
        mse_list.append(my_mse(y_test_fold, y_pred_fold))
    # Returns average r2 and average MSE across all folds
    return np.mean(r2_list), np.mean(mse_list)



In [81]:
kfold_cv(X_train, y_train)

(0.600530729004401, 17.22816186912361)

In [156]:

# Perform grid search using K-fold cross validation to optimize hyperparameters
def grid_search_kf(X_train, y_train, X_test, y_test, param_grid, random_seed=None, K_folds = 5):
    # Create best score and best parameter variables - best score starts as infinity so any real-valued score will be better (lower)
    best_score = float('inf')
    best_params = {}

    # For every combination of parameters in the parameter grid:
    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            for learning_rate in param_grid['learning_rate']:
                # get r2 and MSE from K-fold function tuned to this combination of hyperparameters
                r2, score = kfold_cv(X_train, y_train, n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, random_state=random_seed, K_folds=K_folds)
                # If MSE is better than best_score, replace best_score with this score and existing best_params with these
                if score < best_score:
                    best_score = score
                    best_params = {'n_estimators': n_estimators, 'max_depth': max_depth, 'learning_rate': learning_rate}
    # Initialize model with best parameters
    model = GradientBoostAll(n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'], learning_rate=best_params['learning_rate'])
    # Fit model on training data
    model.fit(X_train, y_train)
    # Model predicts test data
    y_pred = model.predict(X_test)
    # Returns performance metrics from prediction 
    return my_performance(y_test, y_pred)

# Execute grid search
param_grid = {
    'n_estimators': [10, 20, 50, 100],
    'max_depth': [1, 2, 3, 4],
    'learning_rate': [0.01, 0.1, 0.2, 0.5]
}

# Assuming X_train, y_train, X_test, and y_test are already defined
grid_search_kf(X_train, y_train, X_test, y_test, param_grid)


mean squared error: 5.24605434966139
root mean squared error: 2.290426674150777
mean absolute error: 1.7862341568945976
r2 score: 0.865282061596299


In [ ]:
model = GradientBoostAll()

In [147]:
xr, xe, yr, ye = train_test_split(X, y, test_size=0.2, random_state=100)

In [120]:
np.mean(y_train)

20.69327956989247

In [148]:
np.mean(yr)

20.69327956989247